In [1]:
import numpy as np  # Vectorization and arrays
import torch as tc  # PyTorch, used for Tensor operations

import PyPWA as pwa

/home/mark/.anaconda3_install/envs/PyPWA/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class NewMinuitGauss2D(pwa.NestedFunction):
    """
    Minuit 2.0 introduced a new way of working with parameters,
    by passing the values to the calculate function as a numpy array
    instead of has a dictionary of values.

    Using values this way is a little more obtuse, but if you have
    reached the point in your analysis where you are trying to
    squeeze out as much performance from your amplitude as possible,
    swapping to Minuit's array parameters could net you a small
    performance boost at the cost of transparency.
    """

    USE_MP = False
    USE_TORCH = True
    device = ...  # type: tc.device

    def setup(self, array):
        # This example uses Numpy arrays, not Pandas dataframes,
        # so we don't need to convert the values here.

        self.device = tc.device(f"cuda:{self.THREAD}" if self.THREAD >= 0 else "cpu")

        self.__x = tc.from_numpy(array["x"]).to(self.device)
        self.__y = tc.from_numpy(array["y"]).to(self.device)


    def calculate(self, array):
        """
        The old params followed a:
        {"A1": float, "A2": float, "A3": float, "A4": float}
        format.

        The new params are:
        [float, float, float, float]
        which corresponds to:
        [A1, A2, A3, A4]

        """
        scaling = 1 / ( array[1] * array[3])
        left = ((self.__x - array[0])**2)/(array[1]**2)
        right = ((self.__y - array[2])**2)/(array[3]**2)
        return scaling * tc.exp(-(left + right))

In [3]:
# Create basic data with structured numpy arrays
flat_data = np.empty(250_000_000, dtype=[('x', 'f8'), ('y', 'f8')])
flat_data["x"] = np.random.rand(250_000_000) * 20
flat_data["y"] = np.random.rand(250_000_000) * 20

In [4]:
simulation_params = np.array([10, 3, 10, 3])
rejection = pwa.monte_carlo_simulation(
    NewMinuitGauss2D(), flat_data, simulation_params
)
final = flat_data[rejection]

# Simulation uses almost 20Gb of VRAM!
# We really do want to release some of that VRAM back
tc.cuda.empty_cache()

In [5]:
print(
    f"Result length is {len(final)}, "
    f"{(len(final) / 500_000_000) * 100:.2f}% events were kept, "
    f"which is {final.nbytes / 1048576: .2f}Mb in size."
)

Result length is 17669347, 3.53% events were kept, which is  269.61Mb in size.


In [6]:
with pwa.LogLikelihood(NewMinuitGauss2D(), final) as likelihood:
    optimizer = pwa.minuit(np.array([1, 1, 1, 1], float), likelihood)

    for param in [0, 2]:
        optimizer.limits[param] = (.1, None)

    for param in [1, 3]:
        optimizer.limits[param] = (1, None)

    result = optimizer.migrad()
result

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 5.649e+07                  │              Nfcn = 213              │
│ EDM = 4.99e-05 (Goal: 0.0001)    │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │        No Parameters at limit        │
├──────────────────────────────────┼──────────────────────────────────────┤
│ Below EDM threshold (goal x 10)  │           Below call limit           │
├───────────────┬──────────────────┼───────────┬─────────────┬────────────┤
│  Covariance   │     Hesse ok     │ Accurate  │  Pos. def.  │ Not forced │
└───────────────┴──────────────────┴───────────┴─────────────┴────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ x0   │  9.9997   │  0.0005   │            │            │   0.1   │         │       │
│ 1 │ x1   │  3.0000   │  0.0005   │            │            │    1    │         │       │
│ 2 │ x2   │  9.9994   │  0.0005   │            │            │   0.1   │         │       │
│ 3 │ x3   │  3.0002   │  0.0005   │            │            │    1    │         │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌────┬─────────────────────────────────────────┐
│    │        x0        x1        x2        x3 │
├────┼─────────────────────────────────────────┤
│ x0 │  2.55e-07  2.22e-10  7.05e-17 -6.11e-20 │
│ x1 │  2.22e-10  2.55e-07  7.04e-17 -1.41e-16 │
│ x2 │  7.05e-17  7.04e-17  2.55e-07  2.23e-10 │
│ x3 │ -6.11e-20 -1.41e-16  2.23e-10  2.55e-07 │
└────┴─────────────────────────────────────────┘